In [1]:
import pandas as pd
from cassandra.cluster import Cluster

cluster = Cluster(['192.168.7.110', '192.168.7.111', '192.168.7.61'])
#session = cluster.connect('data.storageinfo')
session = cluster.connect()

rows = session.execute('select loc from data.storageinfo limit 5')
for row in rows:
    print(row.loc.split('\u0001'))
    #print(type(row.loc))
    #print(row.loc)

cluster.shutdown()

['192.168.8.71', '80', '/data/form4.pdf', 'HTTP-FsxT6X3DSaAko6L9C1.pdf', 'DATE_OF_BIRTH']
['192.168.8.77', '80', '/test/form1.pdf', 'HTTP-F6zvk04Aac5sNnmbvk.pdf', 'SOCIAL_SECURITY_NUMBER']
['192.168.8.71', '80', '/data/form8.pdf', 'HTTP-FHb6664ub9zFE1qoQ1.pdf', 'ZIP_CODE']
['192.168.8.71', '80', '/data/form7.pdf', 'HTTP-FFhWKCpivhGRyTPPi.pdf', 'SOCIAL_SECURITY_NUMBER']
['192.168.8.71', '80', '/data/form8.pdf', 'HTTP-F8IcJc3mvb6klo2zN4.pdf', 'EMAIL']


In [13]:
from cassandra.cluster import Cluster

def cs_select(cs_host_list, csql_str):
    cluster = Cluster(cs_host_list)
    session = cluster.connect()
    res = session.execute(csql_str)
    cluster.shutdown()
    return res
    
#rows = cs_select(['192.168.7.110', '192.168.7.111'], "select * from api.ds_services where ip='192.168.8.74' and port=1433")
#[print(row.ip, type(row.files), type(row.badges), row.connection, row.port) for row in rows]

DS_DELIMITER = "\u0001"
sel_ip = "192.168.7.71"
sel_port = 1433
sel_l = DS_DELIMITER.join([sel_ip, str(sel_port)]) + DS_DELIMITER
print(sel_l)
sel_u = DS_DELIMITER.join([sel_ip, str(sel_port+1)]) + DS_DELIMITER
print(sel_u)

#query = "select * from scanner.sqlcatalog"# where loc LIKE '%192.168.7.71%'"
#rows = cs_select(['192.168.7.21', '192.168.7.22'], query)
#[':'.join([row.db_name, row.table_name, row.col_name]) for row in rows if sel in row.loc]

query = "select * from scanner.sqlcatalog where loc >= '" + sel_l + "' and loc < '" + sel_u + "' allow filtering"
rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
[':'.join([row.ip, str(row.port), row.db_name, row.table_name, row.col_name]) for row in rows]

#rows = cs_select(['192.168.7.110', '192.168.7.111'], 'select sum(dbs) as dbs, sum(tables) as tables, sum(files) as files from api.ds_services')
#[row.dbs for row in rows][0]

#rows = cs_select(['192.168.7.110', '192.168.7.111'], 'select count(ip) as servers, sum(services) as services from api.ds_servers')
#[row.servers for row in rows][0]
#servers, services = [(row.servers, row.services) for row in rows][0]
#print(servers, services)
#[print(type(row.ip)) for row in rows]

192.168.7.711433
192.168.7.711434


[]

In [31]:
#query = "select * from api.ds_services where ip='192.168.7.155' and port=1433 allow filtering"
query = "select * from api.ds_services where storage_type in (1,2)"

rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
cs_data = [{"ip": row.ip,
            "port": row.port,
            "storage_type": row.storage_type} for row in rows]
print(cs_data)

[{'ip': '192.168.8.71', 'port': 1433, 'storage_type': 1}, {'ip': '192.168.7.155', 'port': 1433, 'storage_type': 1}, {'ip': '192.168.8.74', 'port': 3306, 'storage_type': 2}, {'ip': '192.168.8.71', 'port': 3306, 'storage_type': 2}, {'ip': '192.168.7.74', 'port': 3306, 'storage_type': 2}]


In [3]:
query = "select md5, source_uuid, loc from scanner.hashinfo limit 5"
rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
cs_data = [{"md5": row.md5,
            "source_uuid": row.source_uuid,
            "loc": row.loc} for row in rows]
print(cs_data)

[{'md5': '82161242827b703e6acf9c726942a1e4', 'source_uuid': '9eae4195-b2ae-4d76-83c5-38d29aa1ec83', 'loc': '{"LastScanTime":"0001-01-01T00:00:00Z","Table":"userinfo9","Column":"uid","NormalizedColumn":"uid"}'}, {'md5': '0a09c8844ba8f0936c20bd791130d6b6', 'source_uuid': '9eae4195-b2ae-4d76-83c5-38d29aa1ec83', 'loc': '{"LastScanTime":"0001-01-01T00:00:00Z","Table":"userinfo9","Column":"uid","NormalizedColumn":"uid"}'}, {'md5': 'c4b31ce7d95c75ca70d50c19aef08bf1', 'source_uuid': '9eae4195-b2ae-4d76-83c5-38d29aa1ec83', 'loc': '{"LastScanTime":"0001-01-01T00:00:00Z","Table":"userinfo9","Column":"uid","NormalizedColumn":"uid"}'}, {'md5': '08ced3df2d196884401c4535cae59722', 'source_uuid': '7bc4ef46-93ab-489a-a575-d84da4a72cc9', 'loc': '{"LastScanTime":"0001-01-01T00:00:00Z","Table":"employee","Column":"mail","NormalizedColumn":"email"}'}, {'md5': 'bfc628eb1dc363d228e8d1e9e1e56bfa', 'source_uuid': '7bc4ef46-93ab-489a-a575-d84da4a72cc9', 'loc': '{"LastScanTime":"0001-01-01T00:00:00Z","Table":"em

In [15]:
#query = "select ddn_name, city, count(pii_record) as value from api.content_locations group by ddn_name, city"
query = "select ddn_id, city, count(last_seen_time) as value from api.access_locations group by ddn_id, city"
#query = "select city, pii_record from api.content_locations where ddn_name = 'aaa' allow filtering"
rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
print(cs_data)
cs_data = [{"ddn_name": row.ddn_id,
            "content": row.city,
            "value": row.value} for row in rows]
print(cs_data)
#print(cs_data)
#if cs_data:
#    print("aaa")
#else:
#    print("bbb")
cs_data = [
    {"ddn_name": "hr", "content": "Beijing", "value": 5},
    {"ddn_name": "hr", "content": "Tianjin", "value": 1},
    {"ddn_name": "sales", "content": "Shanghai", "value": 5},
    {"ddn_name": "sales", "content": "Beijing", "value": 22},
    {"ddn_name": "hr", "content": "Nanjing", "value": 1},
]

df = pd.DataFrame(cs_data)
d = df.groupby(['content'])['value'].sum().reset_index().sort_values(by='value', ascending=False).set_index('content').T.to_dict()
#d = d.sort_values(by='value', ascending=False)
print(d)
print(type(d))

#.sort_values(by='value', ascending=False).to_dict()
cs_data = [{"content": k, "value": v['value']} for k, v in d.items()]
cs_data

[]
[{'ddn_name': '290c332d1efec714faa4bb975a075ae8', 'content': 'San Francisco,US', 'value': 3}, {'ddn_name': '54d752f95f5627ffbf66731317db0d20', 'content': 'internal,internal', 'value': 6}, {'ddn_name': '290c332d1efec714faa4bb975a075ae8', 'content': 'internal,internal', 'value': 5}, {'ddn_name': '54d752f95f5627ffbf66731317db0d20', 'content': 'San Francisco,US', 'value': 2}]
{'Beijing': {'value': 27}, 'Shanghai': {'value': 5}, 'Nanjing': {'value': 1}, 'Tianjin': {'value': 1}}
<class 'dict'>


[{'content': 'Beijing', 'value': 27},
 {'content': 'Shanghai', 'value': 5},
 {'content': 'Nanjing', 'value': 1},
 {'content': 'Tianjin', 'value': 1}]

In [5]:
query = "select ddn_name, feature_name as content, feature_value as value from api.pii_special"
cs_data = [{"content": row.content,
            "value": row.value} for row in rows]
cs_data

[]

In [43]:
#query = "select ddn_id as content, bytes as x, frequency as y from api.ddn_vol_freq"
#query = "select ddn_id, ddn_name as content, bytes as x, frequency as y from api.ddn_vol_freq limit 5000"
query = "select ddn_id, tf_loc, storage_type as content, sum(bytes) as x, sum(frequency) as y from api.ddn_vol_freq group by ddn_id, tf_loc, storage_type"

rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
cs_data = [{"content": row.content,
            "x": row.x,
            "y": row.y} for row in rows]
#cs_data = [{"content": row.content,
#            "value": row.value} for row in rows]
print(cs_data)
#cs_data = [
#    {"content": "hr", "x": 5, "y": 5},
#    {"content": "hr", "x": 1, "y": 5},
#    {"content": "sales", "x": 5, "y": 5},
#    {"content": "sales", "x": 2, "y": 5},
#    {"content": "hr", "x": 1, "y": 5},
#]
#df = pd.DataFrame(cs_data)
#dx = df.groupby(['content'])['x'].sum().to_dict()
#dx
#dy = df.groupby(['content'])['y'].sum().to_dict()
#dy
#d = df.groupby(['content'])['x', 'y'].sum().to_dict()
#d
#{'x': {'hr': 7, 'sales': 7}, 'y': {'hr': 15, 'sales': 10}}
#keys = list(d['x'].keys())
#keys
#cs_data = [{"content": k, "value": v} for k, v in d.items()]

#cs_data = [{"content": key, "x": d['x'][key], "y": d['y'][key]} for key in keys]
#cs_data

[{'content': 2, 'x': 697436667904.0, 'y': 8973680}, {'content': 2, 'x': 1097005793280.0, 'y': 14114748}]


In [9]:
#query = "select ip, port from api.ds_services where ip='192.168.7.71' and port=80"
query = "select ip, port from api.ds_services where port=80 allow filtering"
rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
[print(row.ip, row.port) for row in rows]

192.168.8.71 80
192.168.7.71 80


[None, None]

In [10]:
def cs_update(cs_host_list, tab_n, upg_d, sel_d):
    cluster = Cluster(cs_host_list)
    session = cluster.connect()
    upg_str = ",".join(
        [k + "='" + v + "'" if isinstance(v, str) \
         else k + "=" + str(v) if isinstance(v, int) \
         else k + "=" + str(v) \
         for k, v in upg_d.items()])
    sel_str = " and ".join(
        [k + "='" + v + "'" if isinstance(v, str) \
         else k + "=" + str(v) if isinstance(v, int) \
         else k + "=" + str(v) \
         for k, v in sel_d.items()])
    csql_str = "update " + tab_n + " set " + upg_str + " where " + sel_str
    print(csql_str)
    res = session.execute(csql_str)
    session.shutdown()
    cluster.shutdown()
    return res

cs_update(['192.168.7.110', '192.168.7.111', '192.168.7.61'], 
          "api.ds_services", 
          {"connection": False, "status": "Pending"}, 
          {"ip": "192.168.7.71", "port": 80})

update api.ds_services set connection=False,status='Pending' where ip='192.168.7.71' and port=80
